In [ ]:
!pip install datasets -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.6 MB/s eta 0:00:00


In [ ]:
!pip install transformers -qq

In [ ]:
!pip install accelerate -qq

In [ ]:
!pip install fuzzywuzzy -qq

In [ ]:
!pip install python-Levenshtein -qq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.6 MB/s eta 0:00:00


In [ ]:
import random
from typing import Dict, List, Union

In [ ]:
from datasets import load_dataset
from fuzzywuzzy import fuzz, process
from sklearn.metrics import classification_report
from tqdm.notebook import tqdm
from transformers import pipeline

In [ ]:
def prepare_message_for_llm(text: Union[str, List[str]], categories: Dict[str, str]) -> Dict[str, Union[List[Dict[str, str]], List[List[Dict[str, str]]]]]:
    if len(categories) < 2:
        raise RuntimeError(f'The category list is too small! Expected 2 or more categories, got {len(categories)} ones.')
    categories_ = sorted(list(categories.keys()))
    categories_as_string = ', '.join(categories_[:-1]) + ' и ' + categories_[-1]
    if isinstance(text, str):
        prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
                 f'списка тем наиболее представлена в следующем тексте. ' \
                 f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
                 f'Список тем: {categories_as_string}.\n'
        for cur in categories_:
            prompt += f'Текст: {" ".join(categories[cur].split())}\nВаш ответ: {cur}\n'
        prompt += f'Текст: {" ".join(text.split())}\nВаш ответ: '
        messages = [
            {
                'role': 'system',
                'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
            },
            {
                'role': 'user',
                'content': prompt
            }
        ]
    else:
        messages = []
        for it in text:
            prompt = f'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного ' \
                     f'списка тем наиболее представлена в следующем тексте. ' \
                     f'В качестве ответа напишите только название темы из списка, больше ничего.\n' \
                     f'Список тем: {categories_as_string}.\n'
            for cur in categories_:
                prompt += f'Текст: {" ".join(categories[cur].split())}\nВаш ответ: {cur}\n'
            prompt += f'Текст: {" ".join(text.split())}\nВаш ответ: '
            messages.append([
                {
                    'role': 'system',
                    'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.'
                },
                {
                    'role': 'user',
                    'content': prompt
                }
            ])
    return {'message_for_llm': messages}

# Pipeline
## Загрузим модель

In [ ]:
llm_pipeline = pipeline(model='Qwen/Qwen2-7B-Instruct', device_map='auto', torch_dtype='auto')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model-00003-of-00004.safetensors:  27%|##7       | 1.05G/3.86G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.56G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

 ## Загрузим датасет

In [ ]:
DATASET_NAME = 'Davlan/sib200'
DATASET_LANGUAGE = 'rus_Cyrl'
train_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='train')
validation_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='validation')
test_set = load_dataset(DATASET_NAME, DATASET_LANGUAGE, split='test')

README.md:   0%|          | 0.00/47.9k [00:00<?, ?B/s]

data/rus_Cyrl/train.tsv:   0%|          | 0.00/195k [00:00<?, ?B/s]

data/rus_Cyrl/dev.tsv:   0%|          | 0.00/25.3k [00:00<?, ?B/s]

data/rus_Cyrl/test.tsv:   0%|          | 0.00/57.4k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

## Выделим категории

In [ ]:
list_of_categories = sorted(list(
    set(train_set['category']) | set(validation_set['category']) | set(test_set['category'])
))
print(f'Categories for classification are: {list_of_categories}')

Categories for classification are: ['entertainment', 'geography', 'health', 'politics', 'science/technology', 'sports', 'travel']


In [ ]:
print(validation_set)

Dataset({
    features: ['index_id', 'category', 'text'],
    num_rows: 99
})


## Выделим случайные примеры

In [ ]:
examples_by_categories = dict()
for current_category in list_of_categories:
    examples_by_categories[current_category] = random.choice(
        train_set.filter(lambda it: it['category'] == current_category)['text']
    )
    print(f'Category: {current_category}\n')
    print(f'Random text: {examples_by_categories[current_category]}\n\n')

Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Category: entertainment

Random text: Созерцание цветения сакуры, называемое "ханами", вошло в японскую культуру еще в VIII веке.




Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Category: geography

Random text: Спутники, каждый из которых был тяжелее 1 000 фунтов и перемещался со скоростью приблизительно 17 500 миль в час, столкнулись на высоте 491 мили над поверхностью Земли.




Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

...

Category: science/technology

Random text: Для запуска в космос спутника или телескопа необходима гигантская ракета высотой более 100 футов.




Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Category: sports

Random text: Окончательным счётом стала победа в одно очко, 21 к 20, что закончило победную серию All Black в 15-ти играх.




Filter:   0%|          | 0/701 [00:00<?, ? examples/s]

Category: travel

Random text: В некоторых поездах, пересекающих границу, контроль осуществляется во время движения, поэтому, садясь на такой поезд, следует иметь при себе действительное удостоверение личности.




## Обернем тексты в prompt для llm

In [ ]:
validation_set_for_llm = validation_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))
test_set_for_llm = test_set.map(lambda it: prepare_message_for_llm(it['text'], examples_by_categories))

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

Map:   0%|          | 0/204 [00:00<?, ? examples/s]

In [ ]:
print(validation_set_for_llm)

Dataset({
    features: ['index_id', 'category', 'text', 'message_for_llm'],
    num_rows: 99
})


In [ ]:
print(validation_set['text'][0])

Если увеличить расстояние для бега с четверти до половины мили, скорость становится не так важна, тогда как выносливость превращается в абсолютную необходимость.


In [ ]:
print(validation_set_for_llm['message_for_llm'][0])

[{'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного списка тем наиболее представлена в следующем тексте. В качестве ответа напишите только название темы из списка, больше ничего.\nСписок тем: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: Созерцание цветения сакуры, называемое "ханами", вошло в японскую культуру еще в VIII веке.\nВаш ответ: entertainment\nТекст: Спутники, каждый из которых был тяжелее 1 000 фунтов и перемещался со скоростью приблизительно 17 500 миль в час, столкнулись на высоте 491 мили над поверхностью Земли.\nВаш ответ: geography\nТекст: Прочие варианты, которые основаны на биологических ритмах, включают в себя прием жидкости в больших количествах (в частности, воды или чая, известного мочегонного средства) перед сном, что заставляет человека вставать, ч

## Сгенерируем ответы

In [ ]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(validation_set_for_llm['message_for_llm'])
))
y_true = validation_set['category']

  0%|          | 0/99 [00:00<?, ?it/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
print(y_pred[0])

[{'content': 'Вы - полезный помощник, умеющий читать тексты на русском языке, глубоко понимать их и анализировать.', 'role': 'system'}, {'content': 'Прочтите, пожалуйста, следующий текст и определите, какая тема из известного списка тем наиболее представлена в следующем тексте. В качестве ответа напишите только название темы из списка, больше ничего.\nСписок тем: entertainment, geography, health, politics, science/technology, sports и travel.\nТекст: Созерцание цветения сакуры, называемое "ханами", вошло в японскую культуру еще в VIII веке.\nВаш ответ: entertainment\nТекст: Спутники, каждый из которых был тяжелее 1 000 фунтов и перемещался со скоростью приблизительно 17 500 миль в час, столкнулись на высоте 491 мили над поверхностью Земли.\nВаш ответ: geography\nТекст: Прочие варианты, которые основаны на биологических ритмах, включают в себя прием жидкости в больших количествах (в частности, воды или чая, известного мочегонного средства) перед сном, что заставляет человека вставать, ч

In [ ]:
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

                    precision    recall  f1-score   support

           culture       0.00      0.00      0.00         0
     entertainment       0.67      0.67      0.67         9
         geography       0.78      0.88      0.82         8
            health       0.82      0.82      0.82        11
          politics       0.93      0.93      0.93        14
science/technology       0.91      0.84      0.87        25
          security       0.00      0.00      0.00         0
            sports       0.92      1.00      0.96        12
           traffic       0.00      0.00      0.00         0
            travel       0.88      0.70      0.78        20
          wildlife       0.00      0.00      0.00         0

          accuracy                           0.83        99
         macro avg       0.54      0.53      0.53        99
      weighted avg       0.86      0.83      0.84        99



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


 ## Постобработка текста

In [ ]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    y_pred
))

In [ ]:
print(y_pred_with_normalization[0])

sports


In [ ]:
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.67      0.67      0.67         9
         geography       0.78      0.88      0.82         8
            health       0.82      0.82      0.82        11
          politics       0.87      0.93      0.90        14
science/technology       0.91      0.84      0.87        25
            sports       0.86      1.00      0.92        12
            travel       0.83      0.75      0.79        20

          accuracy                           0.84        99
         macro avg       0.82      0.84      0.83        99
      weighted avg       0.84      0.84      0.84        99



## Тестовые данные

In [ ]:
y_pred = list(map(
    lambda x: llm_pipeline(x, max_new_tokens=10)[0]['generated_text'],
    tqdm(test_set_for_llm['message_for_llm'])
))
y_true = test_set['category']

  0%|          | 0/204 [00:00<?, ?it/s]

In [ ]:
print(classification_report(y_true=y_true, y_pred=[x[-1]['content'] for x in y_pred]))

                    precision    recall  f1-score   support

           animals       0.00      0.00      0.00         0
               art       0.00      0.00      0.00         0
    communications       0.00      0.00      0.00         0
     entertainment       0.87      0.68      0.76        19
         geography       0.74      0.82      0.78        17
            health       0.83      0.91      0.87        22
       immigration       0.00      0.00      0.00         0
        literature       0.00      0.00      0.00         0
             media       0.00      0.00      0.00         0
             music       0.00      0.00      0.00         0
          politics       0.94      0.97      0.95        30
          religion       0.00      0.00      0.00         0
science/technology       0.90      0.86      0.88        51
            sports       0.88      0.84      0.86        25
        technology       0.00      0.00      0.00         0
         transport       0.00      0.00

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
y_pred_with_normalization = list(map(
    lambda it: process.extractOne(it[-1]['content'], list_of_categories, scorer=fuzz.token_sort_ratio)[0],
    y_pred
))

In [ ]:
print(classification_report(y_true=y_true, y_pred=y_pred_with_normalization))

                    precision    recall  f1-score   support

     entertainment       0.76      0.68      0.72        19
         geography       0.74      0.82      0.78        17
            health       0.71      0.91      0.80        22
          politics       0.88      0.97      0.92        30
science/technology       0.90      0.88      0.89        51
            sports       0.81      0.84      0.82        25
            travel       0.94      0.72      0.82        40

          accuracy                           0.84       204
         macro avg       0.82      0.83      0.82       204
      weighted avg       0.85      0.84      0.84       204

